# the goal of this notebook is to overview how I use the CSWtask object in CSW.py

In [1]:
import torch as tr
import numpy as np
from CSW import *

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

### sampling a path

In [2]:
# upon creation, graph is initialized
task = CSWTask(graph_pr = 1.0)
task.graph

{0: {1: 0.5, 2: 0.5},
 1: {3: 1.0, 4: 0.0},
 2: {3: 0.0, 4: 1.0},
 3: {5: 1.0, 6: 0.0},
 4: {5: 0.0, 6: 1.0},
 5: {7: 1.0},
 6: {7: 1.0}}

In [3]:
# this method samples a path from the above graph, 
# starting at zero, selecting nodes according to the corresponding probability
task.sample_path()
# note this path does not include the schema cuing feature (see below)

array([0, 2, 4, 6, 7])

### curriculum is manipulated in the training loop

In [4]:
# to simulate experimental results I use two tasks with mirror opposite graphs:
task1 = CSWTask(graph_pr = 1.0)
task2 = CSWTask(graph_pr = 0)

# to manipulate curriculum during training, I switch between these
taskL = [task1,task2]
filleridL = [10,11]

# example training loop wrapper
num_trials = 200
# on each epoch, the LSTM gets a full story, 
# this corresponds to sampling a single path from the graph
# during training, curriculum is manipulated by switching
# what graph the path is being sampled from
block_len = 40 ## 40=blocked, 1=interleaved
task_idx = 0
for epoch in range(num_trials):
  if epoch%block_len==0:
    task_idx = (task_idx+1)%2
    print('\ntask_graph =',task_idx)
    task = taskL[task_idx]
    filler_id = filleridL[task_idx]
  path = task.sample_path()
  ## insert schema cue and format path into data for neural network 
  xtrain,ytrain = task.dataset_onestory_with_marker(
                  path=path,
                  filler_id=filler_id,
                  depth=1)
  print('x=',xtrain[:,0])
  print('y=',ytrain[:,0])
  print()


task_graph = 1
x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  2,  3,  6])
y= tensor([11,  2,  3,  6,  7])

x= tensor([ 0, 11,  1,  4,  5])
y= tensor([11,  1,  4,  5,  7])

x= tensor

# note that the data are integers. quick summary of the network processing steps: (i) the x-integer is transformed into a one-hot vector, (ii) the y-integer provides the label for what output (softmax) unit needs to be maximally active